Reading data from storage

In [0]:
cf = spark.read.format("csv").option(
    "header", True
).load(
    "/Volumes/hexaware_training_workspace_3/default/customer_orders_insight/cleaned_data/customers_cleaned.csv"
)
cf.printSchema()

df = spark.read.format("csv").option(
    "header", True
).load(
    "/Volumes/hexaware_training_workspace_3/default/customer_orders_insight/cleaned_data/deliveries_cleaned.csv"
)
df.printSchema()

of = spark.read.format("csv").option(
    "header", True
).load(
    "/Volumes/hexaware_training_workspace_3/default/customer_orders_insight/cleaned_data/orders_cleaned.csv"
)
of.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- email: string (nullable = true)
 |-- phone: string (nullable = true)

root
 |-- _c0: string (nullable = true)
 |-- status_id: string (nullable = true)
 |-- order_id: string (nullable = true)
 |-- status: string (nullable = true)
 |-- status_time: string (nullable = true)
 |-- remarks: string (nullable = true)

root
 |-- _c0: string (nullable = true)
 |-- order_id: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- order_date: string (nullable = true)
 |-- total_amount: string (nullable = true)
 |-- delivery_date: string (nullable = true)



In [0]:
ef=spark.read.option("header",True).csv("/Volumes/hexaware_training_workspace_3/default/customer_orders_insight/cleaned_data/extracted_delays.csv")
ef.show()

+---+---------+--------+-----------+-------------+------------+-------+
|_c0|status_id|order_id|customer_id|delivery_date|delayed_Days|delayed|
+---+---------+--------+-----------+-------------+------------+-------+
|  2|        3|    1001|          1|   2025-07-03|           2|      1|
|  5|        6|    1002|          2|   2025-07-05|           3|      1|
|  8|        9|    1003|          3|   2025-07-07|           2|      1|
| 11|       12|    1004|          1|   2025-07-09|           2|      1|
| 14|       15|    1005|          4|   2025-07-12|           6|      1|
| 17|       18|    1006|          2|   2025-07-13|           1|      1|
| 20|       21|    1007|          5|   2025-07-14|           6|      1|
| 23|       24|    1008|          3|   2025-07-16|           1|      1|
| 26|       27|    1009|          4|   2025-07-17|           2|      1|
| 29|       30|    1010|          5|   2025-07-18|           1|      1|
+---+---------+--------+-----------+-------------+------------+-

In [0]:
cf.show()
df.show()
of.show(100)

+---+-----------+-------------+--------------------+--------+
|_c0|customer_id|         name|               email|   phone|
+---+-----------+-------------+--------------------+--------+
|  0|          1|Alice Johnson|alice.johnson@exa...|555-0101|
|  1|          2|    Bob Smith|bob.smith@example...|555-0202|
|  2|          3|    Carol Lee|carol.lee@example...|555-0303|
|  3|          4|  David Patel|david.patel@examp...|555-0404|
|  4|          5|     Eva Wang|eva.wang@example.com|555-0505|
+---+-----------+-------------+--------------------+--------+

+---+---------+--------+---------+-------------------+--------------------+
|_c0|status_id|order_id|   status|        status_time|             remarks|
+---+---------+--------+---------+-------------------+--------------------+
|  0|        1|    1001|  pending|2025-07-02 08:00:00|      left warehouse|
|  1|        2|    1001|  shipped|2025-07-03 09:30:00|  departed warehouse|
|  2|        3|    1001|delivered|2025-07-05 14:15:00|  signe

preparing gold view data
and preparing delta tables


In [0]:

joined_table=df.join(ef,on='status_id',how='left').select(df.order_id.alias('order_id'),df.status_id.alias("status_id"),"status","status_time","remarks","customer_id","delivery_date","delayed_Days","delayed")
joined_table=joined_table.fillna("0",subset=["delayed_Days","delayed","customer_id","delivery_date"])
joined_table.show(40)
joined_table.printSchema()

+--------+---------+---------+-------------------+--------------------+-----------+-------------+------------+-------+
|order_id|status_id|   status|        status_time|             remarks|customer_id|delivery_date|delayed_Days|delayed|
+--------+---------+---------+-------------------+--------------------+-----------+-------------+------------+-------+
|    1001|        1|  pending|2025-07-02 08:00:00|      left warehouse|          0|            0|           0|      0|
|    1001|        2|  shipped|2025-07-03 09:30:00|  departed warehouse|          0|            0|           0|      0|
|    1001|        3|delivered|2025-07-05 14:15:00|  signed by customer|          1|   2025-07-03|           2|      1|
|    1002|        4|  pending|2025-07-04 10:00:00|      order received|          0|            0|           0|      0|
|    1002|        5|  shipped|2025-07-05 11:00:00|picked up by carrier|          0|            0|           0|      0|
|    1002|        6|  delayed|2025-07-08 16:00:0

In [0]:
joined_table.write.format("delta").mode("overwrite").save("/Volumes/hexaware_training_workspace_3/default/customer_orders_insight/gold_order_Data/orders")

creating sample data for updation and insertion

In [0]:
csv_data="""order_id,status_id,status,status_time,remarks,customer_id,delivery_date,delayed_Days,delayed
1010,30,not-delivered,2025-07-19 16:45:00,package stolen,5,2025-07-18,1,1
1010,31,delivered,2025-07-25 16:45:00,package delivered to customer in-hand,5,2025-07-25,7,1"""
with open("/Volumes/hexaware_training_workspace_3/default/customer_orders_insight/updation_Data/new_data.csv","w")as f:
    f.write(csv_data)


In [0]:
from pyspark.sql.functions import to_timestamp
new_data=spark.read.option("header",True).csv("/Volumes/hexaware_training_workspace_3/default/customer_orders_insight/updation_Data/new_data.csv")
new_data.show()
new_data=new_data.withColumn('status_time',to_timestamp("status_time"))
new_data=new_data.withColumn('delivery_date',to_timestamp("delivery_date"))
#new_data.write.format("delta").mode("overwrite").save("/Volumes/hexaware_training_workspace_3/default/#customer_orders_insight/updation_Data/new_data_delta")


+--------+---------+-------------+-------------------+--------------------+-----------+-------------+------------+-------+
|order_id|status_id|       status|        status_time|             remarks|customer_id|delivery_date|delayed_Days|delayed|
+--------+---------+-------------+-------------------+--------------------+-----------+-------------+------------+-------+
|    1010|       30|not-delivered|2025-07-19 16:45:00|      package stolen|          5|   2025-07-18|           1|      1|
|    1010|       31|    delivered|2025-07-25 16:45:00|package delivered...|          5|   2025-07-25|           7|      1|
+--------+---------+-------------+-------------------+--------------------+-----------+-------------+------------+-------+



In [0]:
from delta.tables import DeltaTable
deltaTable = DeltaTable.forPath(spark,"/Volumes/hexaware_training_workspace_3/default/customer_orders_insight/gold_order_Data/orders")
deltaTable.alias("old").merge(new_data.alias("new"),"old.status_id=new.status_id").whenMatchedUpdateAll().whenNotMatchedInsertAll().execute()
deltaTable.toDF().show(45)
deltaTable.toDF().write.format("delta").mode("overwrite").save("/Volumes/hexaware_training_workspace_3/default/customer_orders_insight/final_data/updated_data_delta")

+--------+---------+-------------+-------------------+--------------------+-----------+-------------------+------------+-------+
|order_id|status_id|       status|        status_time|             remarks|customer_id|      delivery_date|delayed_Days|delayed|
+--------+---------+-------------+-------------------+--------------------+-----------+-------------------+------------+-------+
|    1001|        1|      pending|2025-07-02 08:00:00|      left warehouse|          0|                  0|           0|      0|
|    1001|        2|      shipped|2025-07-03 09:30:00|  departed warehouse|          0|                  0|           0|      0|
|    1001|        3|    delivered|2025-07-05 14:15:00|  signed by customer|          1|         2025-07-03|           2|      1|
|    1002|        4|      pending|2025-07-04 10:00:00|      order received|          0|                  0|           0|      0|
|    1002|        5|      shipped|2025-07-05 11:00:00|picked up by carrier|          0|          

top 5 delayed customers

In [0]:
deltaTable.toDF().write.format("delta").mode("overwrite").save("dbfs:/FileStore/shared_uploads/delta_table_temp")

In [0]:
spark.sql("""use catalog hive_metastore""")
spark.sql("""create database if not exists customer_order_insight""")
spark.sql("""use customer_order_insight""")
spark.sql("""create table if not exists orders using delta location "dbfs:/FileStore/shared_uploads/delta_table_temp" """)
spark.sql("""select order_id,max(delayed_Days)as total_delay from orders where delayed_Days>0 group by order_id order by total_delay desc limit 5""").show()

+--------+-----------+
|order_id|total_delay|
+--------+-----------+
|    1010|          7|
|    1005|          6|
|    1007|          6|
|    1002|          3|
|    1001|          2|
+--------+-----------+

